In [195]:
import re
import requests
import pendulum

from pydantic import BaseModel
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [89]:
base_url = 'http://fapl.ru'

In [198]:
def get_months_to_iterate() -> list[str]:
    current_date = datetime.now()
    n_months_to_iterate: int = 1
    
    months = []
    for _ in range(n_months_to_iterate):
        months.append(current_date.strftime("%Y/%m/"))
        current_date -= relativedelta(months=1)
    
    return months


def get_soup_from_url(url: str) -> BeautifulSoup:
    response = requests.get(url)
    html = response.content.decode('windows-1251', errors='replace') 
    
    return BeautifulSoup(html, 'html.parser')

In [184]:
months = get_months_to_iterate()

In [193]:
def get_daily_posts_links(month: str) -> list[str]:
    url = base_url + '/calendar/' + month

    response = requests.get(url)
    html = response.content.decode('windows-1251', errors='replace') 
    
    soup = BeautifulSoup(html, 'html.parser')
    
    paragraphs = soup.findAll('p')
    timestamp_pattern = re.compile(r"^\d{2}\.\d{2}\.\d{4}")
    
    daily_posts_links = [
        base_url + p.find('a')['href'] for p in paragraphs
        if timestamp_pattern.match(p.text.split()[0].strip())
        and p.find('a', href=True)
    ]
    
    return daily_posts_links

In [172]:
from pydantic import BaseModel
from datetime import datetime


class Post(BaseModel):
    uid: int
    title: str
    text_content: list[str]
    tags: list[str]
    n_visits: int
    author: str
    date: datetime


def process_post(soup: BeautifulSoup, uid: int) -> Post:
    block = soup.find('div', class_='block')

    title = block.find('h2').text
    text_content_list = [elem.text.strip() for elem in block.find('div', class_='content').findAll('p')]
    tags = [elem.text for elem in block.find('p', class_='tags').findAll('a')]
    
    n_visits = int(re.findall(r'\d+', block.find('p', class_='visits').text)[0])
    author = block.find('p', class_='author').text.strip()
    date = datetime.strptime(block.find('p', class_='date').text.strip(), '%d.%m.%Y %H:%M')

    return Post(uid=uid,
               title=title,
               text_content=text_content_list,
               tags=tags,
               n_visits=n_visits,
               author=author,
               date=date)

In [196]:
from tqdm import tqdm


def get_posts() -> list[Post]:
    months = get_months_to_iterate()
    posts = []
    
    for month in months:
        daily_posts_links = get_daily_posts_links(month)
    
        for post_url in tqdm(daily_posts_links):
            uid = int(re.findall(r'\d+', post_url)[0])
            soup = get_soup_from_url(url=post_url)
    
            post = process_post(soup, uid)
    
            posts.append(post)
            
    return posts

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 115/115 [00:50<00:00,  2.27it/s]


In [211]:
posts[0].model_dump()

{'uid': 112158,
 'title': 'Невилл и Каррагер составили символические сборные первой половины сезона',
 'text_content': ['Эксперты Sky Sports Гари Невилл и Джейми Каррагер составили по символической сборной первой половины этого сезона Премьер-Лиги.',
  'Матч между "Брентфордом" и "Арсеналом" 1 января ознаменует экватор сезона Премьер-Лиги 2024/25, если не учитывать отложенное дерби между "Эвертоном" и "Ливерпулем".\n\r\nУчитывая приближение этого рубежа, во время своего последнего эфира Невилл и Каррагер назвали по 11 игроков, которых они считают лучшими в первой половине этой кампании.\n\r\nВ своих версиях Невилл и Каррагер разошлись только по трем игрокам из оборонительной линии. Гари предпочел Трента Александер-Арнольда, Вильяма Салиба и Йошко Гвардиола, Джейми — Олу Айну, Николу Миленковичу и Энтони Робинсона.'],
 'tags': ['Каррагер', 'Невилл', 'Премьер-Лига'],
 'n_visits': 4822,
 'author': 'mihajlo',
 'date': datetime.datetime(2025, 1, 1, 0, 5)}